In [1]:
import requests, praw, re, time
from bs4 import BeautifulSoup 
import pandas as pd 
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from config import indeed_query,r_client_id,r_secret_key,r_dev,r_username,r_pw

In [5]:
reddit = praw.Reddit(
    client_id=r_client_id,
    client_secret=r_secret_key,
    password=r_pw,
    user_agent=f'testscript by u/{r_dev}',
    username=r_dev,
)
print(reddit.user.me())

Jazzy_Danger_1230


In [11]:
for submission in reddit.subreddit("wallstreetbets").hot(limit=1):
    print(submission.title)
    submission.comment_sort = 'new'
    for comment in submission.comments:
        try:
            if 'BB' in comment.body:
                print(comment.body)
        except:
            pass

Daily Discussion Thread for June 02, 2021
AMC is actually influencing BB
Somone convince me not to yolo BB
DO NOT SELL BB YA APES

&#x200B;

Hold it.
BB time for you to shine
BB SALE THIS SHIT GONNA MOON
Bots playing? BB following every move of AMC just on a smaller scale
BB

CRSR



These are they plays to get into NOW.
BB: 🛵
AMC: 🚀
I thought we were all buying BB>?
Time to rotate into BB , no restrictions there yet...
I bought calls on BB for in 2 weeks, let’s see 😈
What is love, BBBY don’t hurt me, don’t hurt me, no more
I bought BB. Lets see what happens
Do I take profit on BB?? Downvote for No ty
BB is so undervalued
If BB is up 20% in a day. 7 days gonna be 140% ✔️
Middle class getting rich by holding strong together AMC and BB
$BB & $SNDL WILL PRINT 

APES STRONG TOGETHER...
HOLD AND BUY, APES HOLD AND BUY $AMC $BB
AMC APES:

Don't forget to take out your initial investment!

Put that initial investment in BB and start it all over again!
Is BB a buy and hold? Or should I go to A

In [ ]:
p=0
url = f'https://washingtondc.craigslist.org/d/bicycles/search/bia?s={p}'
soup = BeautifulSoup(requests.get(url).text, "html.parser")

   
    

In [ ]:
 els = soup.select('.jobsearch-SerpJobCard') 